In [206]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from tensorflow.keras.layers import Dropout
from tensorflow.keras import utils, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

data = pd.read_csv('./train.csv', encoding = 'big5')

In [208]:
n = data.shape[0]  #n=891
a = data.iloc[:, 2].to_numpy().reshape(n,1)  #Pclass
b = data.iloc[:, 4:8].to_numpy()  #Sex, Age, SibSp, Parch, Ticket

c = data.iloc[:, 9].to_numpy().reshape(n,1)  #Fare
cabin = data.iloc[:,10].to_numpy().reshape(n,1)  #Cabin

for j in range(cabin.shape[0]):
    if cabin[j,0] is not np.nan:
        cabin[j,0] = 2
    else:
        cabin[j,0] = 1
    

d = data.iloc[:, 11].to_numpy().reshape(n, 1)  #Embarked
train_label = data.iloc[:,1]

train = np.concatenate([a,b,c,cabin,d],axis = 1)
train[train == 'male'] = 1
train[train == 'female'] = 2
train[train == 'S'] = 1
train[train == 'C'] = 2
train[train == 'Q'] = 3

train = train.astype ('float32')

for i in range(train.shape[1]):
    col = train[:,i]
    col[np.isnan(col)] = 1
    
train_label = train_label.to_numpy()
train_label = train_label.astype ('float32').reshape(1, -1)  #Survived 

trainOneHot = utils.to_categorical (train_label.flatten())

#test = train_label.flatten()
#print(test.shape)
#print(trainOneHot)
#print(train.shape)
#print(train.T.shape)
print(train_label.shape)
print(trainOneHot.shape)

(1, 891)
(891, 2)


In [214]:
# MLP model
model = Sequential ( )
model.add ( Dense (512, input_dim=8, activation='relu') )
#model.add ( Dense (32, activation='relu') )
model.add ( Dense (2, activation='softmax') )
model.compile (loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary ( )

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 512)               4608      
_________________________________________________________________
dense_73 (Dense)             (None, 2)                 1026      
Total params: 5,634
Trainable params: 5,634
Non-trainable params: 0
_________________________________________________________________


In [226]:
MLP_history=model.fit(train, trainOneHot, validation_split=0.2, epochs=1000, batch_size=128)

Epoch 1/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.2595 - accuracy: 0.9017 - val_loss: 0.4834 - val_accuracy: 0.8324
Epoch 2/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2632 - accuracy: 0.9017 - val_loss: 0.4760 - val_accuracy: 0.8436
Epoch 3/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2613 - accuracy: 0.9017 - val_loss: 0.4727 - val_accuracy: 0.8492
Epoch 4/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2616 - accuracy: 0.8876 - val_loss: 0.5216 - val_accuracy: 0.8268
Epoch 5/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2697 - accuracy: 0.8961 - val_loss: 0.4916 - val_accuracy: 0.8492
Epoch 6/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2742 - accuracy: 0.8961 - val_loss: 0.5215 - val_accuracy: 0.8324
Epoch 7/1000
6/6 [==============================] - 0s 4ms/step - loss: 0.2632 - accuracy: 0.8989 - val_loss: 0.4826 - val_accuracy: 0.8436
Epoch 8/1000
6/6 [==

In [227]:
test_data = pd.read_csv('./test.csv', encoding = 'big5')
m = test_data.shape[0]
a = test_data.iloc[:,1].to_numpy().reshape(m,1)
b = test_data.iloc[:,3:7].to_numpy()

c = test_data.iloc[:,8].to_numpy().reshape(m,1)
cabin = test_data.iloc[:,9].to_numpy().reshape(m,1)

for j in range(cabin.shape[0]):
    if cabin[j,0] is not np.nan:
        cabin[j,0] = 2
    else:
        cabin[j,0] = 1
    

d = test_data.iloc[:,10].to_numpy().reshape(m,1)

test = np.concatenate([a,b,c,cabin,d],axis = 1)
test[test == 'male'] = 1
test[test == 'female'] = 2
test[test == 'S'] = 1
test[test == 'C'] = 2
test[test == 'Q'] = 3

test = test.astype ('float32')

for i in range(test.shape[1]):
    col = test[:,i]
    col[np.isnan(col)] = 1

    

In [228]:
result = model.predict(test)
p = np.zeros((1,m))

for i in range(result.shape[0]):
    if result[i, 0] > result[i, 1]:
        p[0,i] = 0
    else:
        p[0,i] = 1

print(result.shape)
print(p)
#print(result)            
#print(result.shape[0])
#print(p.shape)

(418, 2)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0.
  1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0.
  0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0.
  1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
  1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
  0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
  1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.
  0. 0. 1. 1. 1. 0. 0. 0. 0. 

In [230]:
pid = test_data.iloc[:,0].to_numpy()
#print(pid)
pid0 = int(pid[0])
print(pid0)
pid_n = int(pid.shape[0])
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['PassengerId', 'Survived']
    print(header)
    csv_writer.writerow(header)
    for i in range(pid_n):

        row =  [str(pid0+i), int(p[0][i])]
        csv_writer.writerow(row)

892
['PassengerId', 'Survived']
